In [1]:
import os
import sys
sys.path.append(os.path.abspath(os.path.join('..', 'src')))

from conv_transformer import ConvTransformerModel
from cnn_lstm import ParallelCNNLSTMModel
from ann_baseline import Baseline_ANN
from cnn_lstm import CNN_LSTM_Model, ParallelCNNLSTMModel, LSTM_CNN,ParallelCNNTransformerModel
from cnn_model import CNN_Ieeg_Model
from lstm_model import LSTM_Ieeg_Model


from utils import get_loaders, import_checkpoint, save_checkpoint
import torch
import multiprocessing
import mlflow
import mlflow.pytorch
import torch
import torch.optim as optim
from sklearn.metrics import precision_recall_fscore_support, accuracy_score, confusion_matrix,roc_auc_score
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
import psutil
from torch.cuda.amp import GradScaler, autocast
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import math


# MLflow Setup

In [2]:
os.environ['AWS_ACCESS_KEY_ID'] = 'dIgexhE2iDrGls2qargL'
os.environ['AWS_SECRET_ACCESS_KEY'] = 'IzEzgQpztotDnrIInJdUfUIYngpjJoT18d0FDZf7'
os.environ['MLFLOW_S3_ENDPOINT_URL'] = 'http://localhost:9000'
os.environ['MLFLOW_S3_IGNORE_TLS'] = 'true'
os.environ["MLFLOW_ENABLE_SYSTEM_METRICS_LOGGING"] = "true"
mlflow.set_tracking_uri("http://localhost:5000")

print('tracking uri:', mlflow.get_tracking_uri())

tracking uri: http://localhost:5000


In [3]:
# Configuration
DATA_DIR = '../data/data_normalized_exp2'
DATA_DIR_EV = '../data/data_normalized_with_ev'

SEQ_LENGTH = 300
BATCH_SIZE = 200
NUM_EPOCHS = 1
LEARNING_RATE = 0.0001
EXPERIMENT_NAME = "SECOND_IEEG_MODELS_COMP_FINAL"
# RUN_NAME = "CNN"
PIN_MEMORY = True
LOAD_MODEL = False
NUM_WORKERS = multiprocessing.cpu_count()
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
INPUT_SIZE = SEQ_LENGTH
NUM_CLASSES = 4
CHECKPOINTS_PATH = '../models/checkpoints'

# Before Training

In [4]:
def get_model_size(model):
    param_size = 0
    buffer_size = 0
    for param in model.parameters():
        param_size += param.numel() * param.element_size()
    for buffer in model.buffers():
        buffer_size += buffer.numel() * buffer.element_size()
    size_all_mb = (param_size + buffer_size) / 1024 ** 2
    return size_all_mb

In [5]:
def train_model(model: nn.Module, train_loader: DataLoader, val_loader: DataLoader, optimizer: optim.Optimizer, 
                criterion: nn.Module, num_epochs: int, device: torch.device, save_checkpoint_interval: int = 10, 
                early_stopping_patience: int = 15, checkpoint_dir: str = '../models/checkpoints', 
                results_dir: str = '../models/results', accumulation_steps: int = 2,
                cnn=False, model_name='CNN'):
    """
    Train a deep learning model with the given parameters and log metrics to MLflow.

    Args:
        model (nn.Module): The model to train.
        train_loader (DataLoader): DataLoader for the training data.
        val_loader (DataLoader): DataLoader for the validation data.
        optimizer (optim.Optimizer): Optimizer for updating model parameters.
        criterion (nn.Module): Loss function.
        num_epochs (int): Number of epochs to train.
        device (torch.device): Device to use for training (CPU or GPU).
        save_checkpoint_interval (int, optional): Interval for saving checkpoints. Default is 10.
        early_stopping_patience (int, optional): Patience for early stopping. Default is 15.
        checkpoint_dir (str, optional): Directory to save checkpoints. Default is 'checkpoints'.
        results_dir (str, optional): Directory to save results. Default is 'results'.
        accumulation_steps (int, optional): Number of steps to accumulate gradients before updating weights. Default is 2.
        cnn (bool, optional): If True, use CNN mode. Default is False.
        model_name (str, optional): Name of the model for saving checkpoints. Default is 'CNN'.
    """
    scaler = GradScaler()  # For mixed precision training
    best_val_loss = float('inf')  # Track the best validation loss for early stopping
    patience_counter = 0  # Counter for early stopping

    # Ensure results and checkpoint directories exist
    if not os.path.exists(results_dir):
        os.makedirs(results_dir)
    if not os.path.exists(checkpoint_dir):
        os.makedirs(checkpoint_dir)

    train_metrics = []
    val_metrics = []

    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        y_true_train = []
        y_pred_train = []

        progress_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}", unit="batch")
        optimizer.zero_grad()  # Reset gradients at the start of each epoch

        for batch_idx, (inputs, labels) in enumerate(progress_bar):
            inputs, labels = inputs.to(device), labels.to(device)

            with autocast():  # Mixed precision training
                if not cnn:
                    outputs = model(inputs)
                else:
                    outputs, _ = model(inputs)
                    

                loss = criterion(outputs, labels.squeeze())

            scaler.scale(loss).backward()  # Backpropagation

            scaler.step(optimizer)  # Update weights
            scaler.update()
            optimizer.zero_grad()  # Reset gradients after updating weights

            running_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            y_true_train.extend(labels.squeeze().cpu().numpy())
            y_pred_train.extend(predicted.cpu().numpy())

            avg_loss = running_loss / (batch_idx + 1)
            train_accuracy = accuracy_score(y_true_train, y_pred_train)
            precision, recall, f1, _ = precision_recall_fscore_support(y_true_train, y_pred_train, average='weighted', zero_division=0)

            progress_bar.set_postfix(train_loss=avg_loss, train_accuracy=train_accuracy, train_precision=precision, train_recall=recall, train_f1=f1)

        # Log training metrics to MLflow
        mlflow.log_metric("train_loss", avg_loss, step=epoch)
        mlflow.log_metric("train_accuracy", train_accuracy, step=epoch)
        mlflow.log_metric("train_precision", precision, step=epoch)
        mlflow.log_metric("train_recall", recall, step=epoch)
        mlflow.log_metric("train_f1_score", f1, step=epoch)

        # Store training metrics in DataFrame
        train_metrics.append({
            "epoch": epoch + 1,
            "model_name": model_name,
            "train_loss": avg_loss,
            "train_accuracy": train_accuracy,
            "train_precision": precision,
            "train_recall": recall,
            "train_f1": f1
        })

        # Validation step
        model.eval()
        val_loss = 0.0
        y_true_val = []
        y_pred_val = []

        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)

                with autocast():  # Mixed precision inference
                    if not cnn:
                        outputs = model(inputs)
                    else:
                        outputs, _ = model(inputs)
                    loss = criterion(outputs, labels.squeeze())

                val_loss += loss.item()
                _, predicted = torch.max(outputs, 1)
                y_true_val.extend(labels.squeeze().cpu().numpy())
                y_pred_val.extend(predicted.cpu().numpy())

        avg_val_loss = val_loss / len(val_loader)
        val_accuracy = accuracy_score(y_true_val, y_pred_val)
        val_precision, val_recall, val_f1, _ = precision_recall_fscore_support(y_true_val, y_pred_val, average='weighted', zero_division=0)

        # Log validation metrics to MLflow
        mlflow.log_metric("val_loss", avg_val_loss, step=epoch)
        mlflow.log_metric("val_accuracy", val_accuracy, step=epoch)
        mlflow.log_metric("val_precision", val_precision, step=epoch)
        mlflow.log_metric("val_recall", val_recall, step=epoch)
        mlflow.log_metric("val_f1_score", val_f1, step=epoch)

        # Store validation metrics in DataFrame
        val_metrics.append({
            "epoch": epoch + 1,
            "model_name": model_name,
            "val_loss": avg_val_loss,
            "val_accuracy": val_accuracy,
            "val_precision": val_precision,
            "val_recall": val_recall,
            "val_f1": val_f1
        })

        # Update the progress bar with validation metrics
        progress_bar.set_postfix(train_loss=avg_loss, train_accuracy=train_accuracy, train_precision=precision, train_recall=recall, train_f1=f1, val_loss=avg_val_loss, val_accuracy=val_accuracy, val_precision=val_precision, val_recall=val_recall, val_f1=val_f1)

        # Save checkpoint every 'save_checkpoint_interval' epochs
        if (epoch + 1) % save_checkpoint_interval == 0:
            checkpoint_path = os.path.join(checkpoint_dir, f'checkpoint_{model_name}.pth.tar')
            save_checkpoint({'epoch': epoch + 1, 'state_dict': model.state_dict(), 'optimizer': optimizer.state_dict()}, checkpoint_path)
            mlflow.log_artifact(checkpoint_path, artifact_path="checkpoints")

        # Early stopping based on validation loss
        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            patience_counter = 0  # Reset counter if we get a new best validation loss
        else:
            patience_counter += 1

        if patience_counter >= early_stopping_patience:
            print(f"Early stopping at epoch {epoch + 1} due to no improvement in validation loss.")
            mlflow.log_param(f"{model_name}_epochs_actual", epoch + 1)
            break

        # Clear CUDA cache after each epoch
        torch.cuda.empty_cache()

    # Save training and validation metrics as CSV files
    train_metrics_df = pd.DataFrame(train_metrics)
    val_metrics_df = pd.DataFrame(val_metrics)
    train_metrics_df.to_csv(os.path.join(results_dir, f'train_metrics_{model_name}.csv'), index=False)
    val_metrics_df.to_csv(os.path.join(results_dir, f'val_metrics_{model_name}.csv'), index=False)

    # Clear CUDA cache at the end of training
    torch.cuda.empty_cache()

In [6]:

def evaluate_model(model: nn.Module, test_loader: DataLoader, dataset: Dataset, 
                   device: torch.device, img_path: str, results_dir: str,
                   run_name: str, batch_size: int = 16, cnn=False):
    """
    Evaluate a deep learning model and log metrics to MLflow.

    Args:
        model (nn.Module): The model to evaluate.
        test_loader (DataLoader): DataLoader for the test data.
        dataset (Dataset): The dataset containing the test data.
        device (torch.device): Device to use for evaluation (CPU or GPU).
        img_path (str): Path to save the confusion matrix image.
        run_name (str): Name of the MLflow run.
        batch_size (int, optional): Batch size for evaluation. Default is 16.
    """
    model.eval()
    y_true_test = []
    y_pred_test = []
    feature_maps = []

    with torch.no_grad():
        for inputs, labels in tqdm(test_loader, desc="Evaluating", unit="batch"):
            inputs, labels = inputs.to(device), labels.to(device)
            if not cnn: 
                outputs = model(inputs)

            else:
                outputs, feature_map = model(inputs)
                feature_maps.append([fm.cpu() for fm in feature_map])  # Move feature maps to CPU to free GPU memory

            _, predicted = torch.max(outputs, 1)
            y_true_test.extend(labels.squeeze().cpu().numpy())
            y_pred_test.extend(predicted.cpu().numpy())

            # Clear cache to free up memory
            torch.cuda.empty_cache()

    test_accuracy = accuracy_score(y_true_test, y_pred_test)
    precision, recall, f1, _ = precision_recall_fscore_support(y_true_test, y_pred_test, average='weighted', zero_division=0)

    print(f'Accuracy of the model on the test data: {test_accuracy:.2f}%')
    print(f'Precision: {precision:.4f}, Recall: {recall:.4f}, F1 Score: {f1:.4f}')

    mlflow.log_metric("test_accuracy", test_accuracy)
    mlflow.log_metric("test_precision", precision)
    mlflow.log_metric("test_recall", recall)
    mlflow.log_metric("test_f1", f1)

    # Confusion matrix
    cm = confusion_matrix(y_true_test, y_pred_test)
    cm_df = pd.DataFrame(cm, index=dataset.label_encoder.classes_, columns=dataset.label_encoder.classes_)
    plt.figure(figsize=(10, 7))
    sns.heatmap(cm_df, annot=True, fmt='d', cmap='Blues')
    plt.ylabel('Actual')
    plt.xlabel('Predicted')
    plt.title('Confusion Matrix')
    img_file = os.path.join(img_path, f"confusion_matrix_{run_name}.png")
    cm_file = os.path.join(results_dir, f"confusion_matrix_{run_name}.csv")

    plt.savefig(img_file)
    cm_df.to_csv(cm_file)

    mlflow.log_artifact(img_file)
    mlflow.log_artifact(cm_file)

    plt.close()

    # Save feature maps and labels to file
    if cnn:
        feature_maps_file = os.path.join(img_path, f"feature_maps_{run_name}.pt")
        torch.save((feature_maps, y_true_test, y_pred_test), feature_maps_file)
    # mlflow.log_artifact(feature_maps_file, artifact_path="feature_maps")

    return y_true_test, y_pred_test


# ANN with evoked response

In [7]:
train_loader, val_loader, test_loader, dataset = get_loaders(data_dir=DATA_DIR_EV, with_val_loader=True, 
                                                             batch_size=BATCH_SIZE, num_workers=NUM_WORKERS,
                                                             pin_memory=PIN_MEMORY, test_size=0.15, 
                                                             seq_length=SEQ_LENGTH, model_type="mlp")

KeyboardInterrupt: 

In [8]:
hidden_layers = [4096,4096,2048,2048,1024,1024]
model = Baseline_ANN(INPUT_SIZE, NUM_CLASSES, hidden_layers=hidden_layers, dropout=0.3).to(DEVICE)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
model

Baseline_ANN(
  (hidden_layers): ModuleList(
    (0): Linear(in_features=2000, out_features=4096, bias=True)
    (1): Linear(in_features=4096, out_features=4096, bias=True)
    (2): Linear(in_features=4096, out_features=2048, bias=True)
    (3): Linear(in_features=2048, out_features=2048, bias=True)
    (4): Linear(in_features=2048, out_features=1024, bias=True)
    (5): Linear(in_features=1024, out_features=1024, bias=True)
  )
  (dropout): Dropout(p=0.3, inplace=False)
  (activation): ReLU()
  (output_layer): Linear(in_features=1024, out_features=4, bias=True)
)

In [9]:
print(f'Model size: {get_model_size(model):.3f} MB')


Model size: 155.320 MB


In [11]:
for loader_name, loader in zip(['train_loader', 'test_loader', 'val_loader'], [train_loader, test_loader, val_loader]):
    print(f"Checking {loader_name}")
    for i, (inputs, labels) in enumerate(loader):
        print(f"{loader_name} - Batch {i}: inputs shape = {inputs.squeeze(0).shape}, labels shape = {labels.flatten().shape}")
        if inputs.shape[0] != 64:
            print(f"Warning: {loader_name} - Batch {i} has a different batch size: {inputs.shape[0]}, {labels.shape[0]}")


Checking train_loader
train_loader - Batch 0: inputs shape = torch.Size([64, 2000]), labels shape = torch.Size([64])
train_loader - Batch 1: inputs shape = torch.Size([64, 2000]), labels shape = torch.Size([64])
train_loader - Batch 2: inputs shape = torch.Size([64, 2000]), labels shape = torch.Size([64])
train_loader - Batch 3: inputs shape = torch.Size([64, 2000]), labels shape = torch.Size([64])
train_loader - Batch 4: inputs shape = torch.Size([64, 2000]), labels shape = torch.Size([64])
train_loader - Batch 5: inputs shape = torch.Size([64, 2000]), labels shape = torch.Size([64])
train_loader - Batch 6: inputs shape = torch.Size([64, 2000]), labels shape = torch.Size([64])
train_loader - Batch 7: inputs shape = torch.Size([64, 2000]), labels shape = torch.Size([64])
train_loader - Batch 8: inputs shape = torch.Size([64, 2000]), labels shape = torch.Size([64])
train_loader - Batch 9: inputs shape = torch.Size([64, 2000]), labels shape = torch.Size([64])
train_loader - Batch 10: inp

In [10]:
mlflow.set_experiment(EXPERIMENT_NAME)

<Experiment: artifact_location='mlflow-artifacts:/14', creation_time=1718790684663, experiment_id='14', last_update_time=1718790684663, lifecycle_stage='active', name='SECOND_IEEG_MODELS_COMP_FINAL', tags={}>

In [12]:
# Train and Evaluate the Model with MLflow
run_name = "ANN_with_er"
model_name = "ANN_baseline_wr"
results_dir = "../models/results"
with mlflow.start_run(run_name=run_name) as run:
    # Log parameters
    mlflow.log_param("epochs", NUM_EPOCHS)
    mlflow.log_param("batch_size", BATCH_SIZE)
    mlflow.log_param("learning_rate", LEARNING_RATE)
    mlflow.log_param("model", model_name)
    mlflow.log_param("input_size", SEQ_LENGTH)
    mlflow.log_param("num_classes", NUM_CLASSES)
    mlflow.log_dict(dataset.get_class_mapping(), "class_mapping.json")

    # Train and Evaluate the Model
    train_model(model, train_loader,val_loader, optimizer, criterion, NUM_EPOCHS, DEVICE, 
                save_checkpoint_interval=10, checkpoint_dir=CHECKPOINTS_PATH, 
                model_name=model_name, early_stopping_patience=40, cnn=False)
    _, _ = evaluate_model(model, test_loader, dataset, DEVICE, results_dir=results_dir,img_path='../plots', run_name=run_name)

    # Log the model
    mlflow.pytorch.log_model(model, model_name)

2024/06/19 12:59:21 INFO mlflow.system_metrics.system_metrics_monitor: Started monitoring system metrics.


Epoch 1/50:   0%|          | 0/438 [00:00<?, ?batch/s]

Epoch 2/50:   0%|          | 0/438 [00:00<?, ?batch/s]

Epoch 3/50:   0%|          | 0/438 [00:00<?, ?batch/s]

Epoch 4/50:   0%|          | 0/438 [00:00<?, ?batch/s]

...

Epoch 50/50:   0%|          | 0/438 [00:00<?, ?batch/s]

Checkpoint saved successfully.


Evaluating:   0%|          | 0/85 [00:00<?, ?batch/s]

Accuracy of the model on the test data: 0.77%
Precision: 0.7552, Recall: 0.7688, F1 Score: 0.7599


2024/06/19 13:11:20 INFO mlflow.system_metrics.system_metrics_monitor: Stopping system metrics monitoring...
2024/06/19 13:11:20 INFO mlflow.system_metrics.system_metrics_monitor: Successfully terminated system metrics monitoring!


# ANN Baseline

In [7]:
train_loader, val_loader, test_loader, dataset = get_loaders(data_dir=DATA_DIR, with_val_loader=True, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS,
                                        pin_memory=PIN_MEMORY, test_size=0.15, seq_length=SEQ_LENGTH, model_type="mlp")

Total dataset size: 29316
Train indices length: 24918, Test indices length: 4398
Train indices after val split: 22426, Val indices: 2492
Train dataset length: 22426, Val dataset length: 2492, Test dataset length: 4398


In [8]:
hidden_layers = [4096,4096,2048,2048,1024,1024]
model = Baseline_ANN(INPUT_SIZE, NUM_CLASSES, hidden_layers=hidden_layers, dropout=0.3).to(DEVICE)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
model

Baseline_ANN(
  (hidden_layers): ModuleList(
    (0): Linear(in_features=2000, out_features=4096, bias=True)
    (1): Linear(in_features=4096, out_features=4096, bias=True)
    (2): Linear(in_features=4096, out_features=2048, bias=True)
    (3): Linear(in_features=2048, out_features=2048, bias=True)
    (4): Linear(in_features=2048, out_features=1024, bias=True)
    (5): Linear(in_features=1024, out_features=1024, bias=True)
  )
  (dropout): Dropout(p=0.3, inplace=False)
  (activation): ReLU()
  (output_layer): Linear(in_features=1024, out_features=4, bias=True)
)

In [9]:
print(f'Model size: {get_model_size(model):.3f} MB')


Model size: 155.320 MB


In [10]:
mlflow.set_experiment(EXPERIMENT_NAME)

<Experiment: artifact_location='mlflow-artifacts:/14', creation_time=1718790684663, experiment_id='14', last_update_time=1718790684663, lifecycle_stage='active', name='SECOND_IEEG_MODELS_COMP_FINAL', tags={}>

In [11]:
# Train and Evaluate the Model with MLflow
run_name = "ANN"
model_name = "ANN_baseline"
results_dir = "../models/results"
with mlflow.start_run(run_name=run_name) as run:
    # Log parameters
    mlflow.log_param("epochs", NUM_EPOCHS)
    mlflow.log_param("batch_size", BATCH_SIZE)
    mlflow.log_param("learning_rate", LEARNING_RATE)
    mlflow.log_param("model", model_name)
    mlflow.log_param("input_size", SEQ_LENGTH)
    mlflow.log_param("num_classes", NUM_CLASSES)
    mlflow.log_dict(dataset.get_class_mapping(), "class_mapping.json")

    # Train and Evaluate the Model
    train_model(model, train_loader,val_loader, optimizer, criterion, NUM_EPOCHS, DEVICE, 
                save_checkpoint_interval=10, checkpoint_dir=CHECKPOINTS_PATH, 
                model_name=model_name, early_stopping_patience=40, cnn=False)
    _,_ = evaluate_model(model, test_loader, dataset, DEVICE, results_dir=results_dir,img_path='../plots', run_name=run_name)

    # Log the model
    mlflow.pytorch.log_model(model, model_name)

2024/06/19 13:16:23 INFO mlflow.system_metrics.system_metrics_monitor: Started monitoring system metrics.


Epoch 1/50:   0%|          | 0/350 [00:00<?, ?batch/s]

Epoch 2/50:   0%|          | 0/350 [00:00<?, ?batch/s]

Epoch 3/50:   0%|          | 0/350 [00:00<?, ?batch/s]

Epoch 4/50:   0%|          | 0/350 [00:00<?, ?batch/s]

...

Epoch 50/50:   0%|          | 0/350 [00:00<?, ?batch/s]

Checkpoint saved successfully.


Evaluating:   0%|          | 0/68 [00:00<?, ?batch/s]

Accuracy of the model on the test data: 0.72%
Precision: 0.7070, Recall: 0.7171, F1 Score: 0.7112


2024/06/19 13:25:44 INFO mlflow.system_metrics.system_metrics_monitor: Stopping system metrics monitoring...
2024/06/19 13:25:44 INFO mlflow.system_metrics.system_metrics_monitor: Successfully terminated system metrics monitoring!


# CNN

In [7]:
train_loader, val_loader, test_loader, dataset = get_loaders(data_dir=DATA_DIR, with_val_loader=True, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS,
                                        pin_memory=PIN_MEMORY, test_size=0.15, seq_length=SEQ_LENGTH, model_type="cnn")

Total dataset size: 29316
Train indices length: 24918, Test indices length: 4398
Train indices after val split: 22426, Val indices: 2492
Train dataset length: 22426, Val dataset length: 2492, Test dataset length: 4398


In [8]:
model = CNN_Ieeg_Model(SEQ_LENGTH, NUM_CLASSES).to(DEVICE)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
print(f'Model size: {get_model_size(model):.3f} MB')

model

Model size: 252.997 MB


CNN_Ieeg_Model(
  (conv_layers): ModuleList(
    (0): Conv1d(1, 64, kernel_size=(3,), stride=(1,), padding=(1,))
    (1): Conv1d(64, 128, kernel_size=(3,), stride=(1,), padding=(1,))
    (2): Conv1d(128, 256, kernel_size=(3,), stride=(1,), padding=(1,))
    (3): Conv1d(256, 512, kernel_size=(3,), stride=(1,), padding=(1,))
  )
  (bn_layers): ModuleList(
    (0): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (activation): ReLU()
  (dropout): Dropout(p=0.3, inplace=False)
  (maxpool): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc_layers): ModuleList(
    (0): Linear(in_features=64000, out_features=1024, bias=True)
    (1): Linear(in_features=1024, o

In [9]:
mlflow.set_experiment(EXPERIMENT_NAME)

<Experiment: artifact_location='mlflow-artifacts:/14', creation_time=1718790684663, experiment_id='14', last_update_time=1718790684663, lifecycle_stage='active', name='SECOND_IEEG_MODELS_COMP_FINAL', tags={}>

In [10]:
# Train and Evaluate the Model with MLflow
run_name = "CNN"
model_name = "CNN"
results_dir = "../models/results"
with mlflow.start_run(run_name=run_name) as run:
    # Log parameters
    mlflow.log_param("epochs", NUM_EPOCHS)
    mlflow.log_param("batch_size", BATCH_SIZE)
    mlflow.log_param("learning_rate", LEARNING_RATE)
    mlflow.log_param("model", model_name)
    mlflow.log_param("input_size", SEQ_LENGTH)
    mlflow.log_param("num_classes", NUM_CLASSES)
    mlflow.log_dict(dataset.get_class_mapping(), "class_mapping.json")

    # Train and Evaluate the Model
    train_model(model, train_loader,val_loader, optimizer, criterion, NUM_EPOCHS, DEVICE, 
                save_checkpoint_interval=10, checkpoint_dir=CHECKPOINTS_PATH, 
                model_name=model_name, early_stopping_patience=50, cnn=True)
    _,_ = evaluate_model(model, test_loader, dataset, DEVICE, 
                         results_dir=results_dir,
                         img_path='../plots', 
                         run_name=run_name,
                         cnn=True)

    # Log the model
    mlflow.pytorch.log_model(model, model_name)

2024/06/19 17:34:51 INFO mlflow.system_metrics.system_metrics_monitor: Started monitoring system metrics.


Epoch 1/50:   0%|          | 0/350 [00:00<?, ?batch/s]

Epoch 2/50:   0%|          | 0/350 [00:00<?, ?batch/s]

Epoch 3/50:   0%|          | 0/350 [00:00<?, ?batch/s]

Epoch 4/50:   0%|          | 0/350 [00:00<?, ?batch/s]

...

Epoch 50/50:   0%|          | 0/350 [00:00<?, ?batch/s]

Checkpoint saved successfully.


Evaluating:   0%|          | 0/68 [00:00<?, ?batch/s]

Accuracy of the model on the test data: 0.96%
Precision: 0.9624, Recall: 0.9614, F1 Score: 0.9609


2024/06/19 17:50:51 INFO mlflow.system_metrics.system_metrics_monitor: Stopping system metrics monitoring...
2024/06/19 17:50:51 INFO mlflow.system_metrics.system_metrics_monitor: Successfully terminated system metrics monitoring!


# CONV Transformer

In [7]:
train_loader, val_loader, test_loader, dataset = get_loaders(data_dir=DATA_DIR, with_val_loader=True, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS,
                                        pin_memory=PIN_MEMORY, test_size=0.15, seq_length=SEQ_LENGTH, model_type="cnn")

Total dataset size: 205212
Train indices length: 174430, Test indices length: 30782
Train indices after val split: 156987, Val indices: 17443
Train dataset length: 156987, Val dataset length: 17443, Test dataset length: 30782


In [22]:
for loader_name, loader in zip(['train_loader', 'test_loader','val_loader'], [train_loader, test_loader, val_loader]):
    for i, (inputs, labels) in enumerate(loader):
        print(f"{loader_name} - Batch {i}: inputs shape = {inputs.shape}, labels shape = {labels.shape}")
        if i == 0:  # Only print the first batch for brevity
            break

train_loader - Batch 0: inputs shape = torch.Size([200, 1, 300]), labels shape = torch.Size([200])
test_loader - Batch 0: inputs shape = torch.Size([200, 1, 300]), labels shape = torch.Size([200])
val_loader - Batch 0: inputs shape = torch.Size([200, 1, 300]), labels shape = torch.Size([200])


In [8]:
# Model parameters
input_size = SEQ_LENGTH  # Use the sequence length provided by your dataset
num_classes = 4  # Number of classes for classification
transformer_dim = 512  # Smaller transformer dimension
num_heads = 8  # Fewer attention heads
transformer_depth = 7  # Fewer transformer layers
fc_neurons = [512, 256]  # Reduced fully connected layer sizes
fc_transformer = 512
dropout = 0.3  # Dropout rate

In [9]:
model = ConvTransformerModel(
    input_size=input_size,
    num_classes=num_classes,
    transformer_dim=transformer_dim,
    num_heads=num_heads,
    transformer_depth=transformer_depth,
    fc_neurons=fc_neurons,
    fc_transformer=fc_transformer,
    dropout=dropout,
    activation=nn.GELU()
).to(DEVICE)

In [10]:
print(f'Model size: {get_model_size(model):.3f} MB')
model

Model size: 91.161 MB


ConvTransformerModel(
  (conv_embedding_stem): ConvEmbeddingStem(
    (conv1): Conv1d(1, 256, kernel_size=(10,), stride=(2,), padding=(4,), bias=False)
    (act1): GELU(approximate='none')
    (bn1): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv2): Conv1d(256, 512, kernel_size=(3,), stride=(1,), padding=(1,), bias=False)
    (act2): GELU(approximate='none')
    (bn2): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv3): Conv1d(512, 512, kernel_size=(3,), stride=(2,), padding=(1,), bias=False)
    (act3): GELU(approximate='none')
    (bn3): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (transformer_blocks): ModuleList(
    (0-6): 7 x MultiheadSelfAttentionBlock(
      (norm1): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
      (attn): MultiheadAttention(
        (out_proj): NonDynamicallyQuantizableLinear(in_features=512, out_features=512, bias=False)
 

In [11]:
optimizer = optim.Adam(model.parameters(), lr = LEARNING_RATE)
criterion =  nn.CrossEntropyLoss()  

In [12]:
mlflow.set_experiment(EXPERIMENT_NAME)

<Experiment: artifact_location='mlflow-artifacts:/14', creation_time=1718790684663, experiment_id='14', last_update_time=1718790684663, lifecycle_stage='active', name='SECOND_IEEG_MODELS_COMP_FINAL', tags={}>

In [13]:
# Train and Evaluate the Model with MLflow
run_name = "run_CNN_TNN_sl500"
model_name = "CNN_TNN_sl500"
results_dir = "../models/results"
with mlflow.start_run(run_name=run_name) as run:
    # Log parameters
    mlflow.log_param("epochs", NUM_EPOCHS)
    mlflow.log_param("batch_size", BATCH_SIZE)
    mlflow.log_param("learning_rate", LEARNING_RATE)
    mlflow.log_param("model", model_name)
    mlflow.log_param("input_size", SEQ_LENGTH)
    mlflow.log_param("num_classes", NUM_CLASSES)
    mlflow.log_dict(dataset.get_class_mapping(), "class_mapping.json")

    # Train and Evaluate the Model
    train_model(model, train_loader,val_loader, optimizer, criterion, NUM_EPOCHS, DEVICE, 
                save_checkpoint_interval=10, checkpoint_dir=CHECKPOINTS_PATH, 
                model_name=model_name, early_stopping_patience=10, cnn=False)
    _,_ = evaluate_model(model, test_loader, dataset, DEVICE, 
                         results_dir=results_dir,
                         img_path='../plots', 
                         run_name=run_name,
                         cnn=False)

    # Log the model
    mlflow.pytorch.log_model(model, model_name)

2024/06/23 19:36:25 INFO mlflow.system_metrics.system_metrics_monitor: Started monitoring system metrics.


Epoch 1/1:   0%|          | 0/784 [00:00<?, ?batch/s]

/home/danielcrovo/anaconda3/envs/ieeg/lib/python3.12/site-packages/torch/nn/modules/conv.py:306: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv1d(input, weight, bias, self.stride,


Evaluating:   0%|          | 0/153 [00:00<?, ?batch/s]

Accuracy of the model on the test data: 0.93%
Precision: 0.9331, Recall: 0.9342, F1 Score: 0.9331


2024/06/23 19:41:35 INFO mlflow.system_metrics.system_metrics_monitor: Stopping system metrics monitoring...
2024/06/23 19:41:35 INFO mlflow.system_metrics.system_metrics_monitor: Successfully terminated system metrics monitoring!


# CNN + LSTM Parallel

In [7]:
train_loader, val_loader, test_loader, dataset = get_loaders(data_dir=DATA_DIR, with_val_loader=True, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS,
                                        pin_memory=PIN_MEMORY, test_size=0.15, seq_length=SEQ_LENGTH, model_type="cnn")

Total dataset size: 122150
Train indices length: 103827, Test indices length: 18323
Train indices after val split: 93444, Val indices: 10383
Train dataset length: 93444, Val dataset length: 10383, Test dataset length: 18323


In [8]:
model = ParallelCNNLSTMModel(input_size=SEQ_LENGTH, 
                            num_classes=NUM_CLASSES,
                            input_size_lstm=1,
                            # conv_filters=[64,128,256],
                            # fc_neurons=[1024,128],
                            # lstm_hidden_size=64,
                            lstm_num_layers=3
                        ).to(DEVICE)
print(f'Model size: {get_model_size(model):.3f} MB')
model

Model size: 65.848 MB


ParallelCNNLSTMModel(
  (cnn_head): CNN_Head(
    (conv_layers): ModuleList(
      (0): Conv1d(1, 64, kernel_size=(3,), stride=(1,), padding=(1,))
      (1): Conv1d(64, 128, kernel_size=(3,), stride=(1,), padding=(1,))
      (2): Conv1d(128, 256, kernel_size=(3,), stride=(1,), padding=(1,))
      (3): Conv1d(256, 512, kernel_size=(3,), stride=(1,), padding=(1,))
    )
    (bn_layers): ModuleList(
      (0): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (3): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (activation): ReLU()
    (dropout): Dropout(p=0.3, inplace=False)
    (maxpool): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (lstm): LSTM(1, 64, num_layers=3, batch_first=True)
  (fc_lst

In [9]:
optimizer = optim.Adam(model.parameters(), lr = LEARNING_RATE)
criterion =  nn.CrossEntropyLoss() 

In [10]:
experiment = mlflow.get_experiment_by_name(EXPERIMENT_NAME)
if experiment is None:
    experiment_id = mlflow.create_experiment(EXPERIMENT_NAME)
else:
    experiment_id = experiment.experiment_id
    if experiment.lifecycle_stage == 'deleted':
        mlflow.tracking.MlflowClient().restore_experiment(experiment_id)

In [11]:
mlflow.set_experiment(EXPERIMENT_NAME)

<Experiment: artifact_location='mlflow-artifacts:/14', creation_time=1718790684663, experiment_id='14', last_update_time=1718790684663, lifecycle_stage='active', name='SECOND_IEEG_MODELS_COMP_FINAL', tags={}>

In [12]:

# Train and Evaluate the Model with MLflow
run_name = "P_CNN_LSTM_Experiment_sl500"
model_name = "P_CNN_LSTM_sl500"
results_dir = "../models/results"
with mlflow.start_run(run_name=run_name) as run:
    # Log parameters
    mlflow.log_param("epochs", NUM_EPOCHS)
    mlflow.log_param("batch_size", BATCH_SIZE)
    mlflow.log_param("learning_rate", LEARNING_RATE)
    mlflow.log_param("model", model_name)
    mlflow.log_param("input_size", SEQ_LENGTH)
    mlflow.log_param("num_classes", NUM_CLASSES)
    mlflow.log_dict(dataset.get_class_mapping(), "class_mapping.json")

    # Train and Evaluate the Model
    train_model(model, train_loader,val_loader, optimizer, criterion, NUM_EPOCHS, DEVICE, 
                save_checkpoint_interval=10, checkpoint_dir=CHECKPOINTS_PATH, 
                model_name=model_name, early_stopping_patience=10, cnn=False)
    _,_ = evaluate_model(model, test_loader, dataset, DEVICE, 
                         results_dir=results_dir,
                         img_path='../plots', 
                         run_name=run_name,
                         cnn=False)

    # Log the model
    mlflow.pytorch.log_model(model, model_name)

2024/06/20 20:16:46 INFO mlflow.system_metrics.system_metrics_monitor: Started monitoring system metrics.


Epoch 1/50:   0%|          | 0/1460 [00:00<?, ?batch/s]

/home/danielcrovo/anaconda3/envs/ieeg/lib/python3.12/site-packages/torch/autograd/graph.py:744: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


Epoch 2/50:   0%|          | 0/1460 [00:00<?, ?batch/s]

Epoch 3/50:   0%|          | 0/1460 [00:00<?, ?batch/s]

...

Early stopping at epoch 38 due to no improvement in validation loss.


Evaluating:   0%|          | 0/286 [00:00<?, ?batch/s]

/home/danielcrovo/anaconda3/envs/ieeg/lib/python3.12/site-packages/torch/nn/modules/conv.py:306: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv1d(input, weight, bias, self.stride,


Accuracy of the model on the test data: 0.97%
Precision: 0.9690, Recall: 0.9689, F1 Score: 0.9688


2024/06/20 21:03:59 INFO mlflow.system_metrics.system_metrics_monitor: Stopping system metrics monitoring...
2024/06/20 21:03:59 INFO mlflow.system_metrics.system_metrics_monitor: Successfully terminated system metrics monitoring!


# LSTM 

In [7]:
train_loader, val_loader, test_loader, dataset = get_loaders(data_dir=DATA_DIR, with_val_loader=True, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS,
                                        pin_memory=PIN_MEMORY, test_size=0.15, seq_length=SEQ_LENGTH, model_type="seq")

Total dataset size: 29316
Train indices length: 24918, Test indices length: 4398
Train indices after val split: 22426, Val indices: 2492
Train dataset length: 22426, Val dataset length: 2492, Test dataset length: 4398


In [8]:
model = LSTM_Ieeg_Model(device=DEVICE,
                        input_size=1, 
                        num_classes=NUM_CLASSES, 
                        lstm_layers=8, 
                        lstm_h_size=380,
                        fc_neurons=[2048,512], 
                        bidirectional=False).to(DEVICE)
print(f'Model size: {get_model_size(model):.3f} MB')
model

Model size: 40.135 MB


LSTM_Ieeg_Model(
  (activation): ReLU()
  (lstm): LSTM(1, 380, num_layers=8, batch_first=True, dropout=0.1)
  (fc_layers): ModuleList(
    (0): Linear(in_features=380, out_features=2048, bias=True)
    (1): Linear(in_features=2048, out_features=512, bias=True)
  )
  (output_layer): Linear(in_features=512, out_features=4, bias=True)
  (dropout_layer): Dropout(p=0.1, inplace=False)
)

In [9]:
optimizer = optim.Adam(model.parameters(), lr = LEARNING_RATE)
criterion =  nn.CrossEntropyLoss()  

In [10]:
mlflow.set_experiment(EXPERIMENT_NAME)

<Experiment: artifact_location='mlflow-artifacts:/14', creation_time=1718790684663, experiment_id='14', last_update_time=1718790684663, lifecycle_stage='active', name='SECOND_IEEG_MODELS_COMP_FINAL', tags={}>

In [11]:
run_name = "run_2_LSTM_2000"
model_name = "LSTM_2000"
results_dir = "../models/results"
with mlflow.start_run(run_name=run_name) as run:
    # Log parameters
    mlflow.log_param("epochs", NUM_EPOCHS)
    mlflow.log_param("batch_size", BATCH_SIZE)
    mlflow.log_param("learning_rate", LEARNING_RATE)
    mlflow.log_param("model", model_name)
    mlflow.log_param("input_size", SEQ_LENGTH)
    mlflow.log_param("num_classes", NUM_CLASSES)
    mlflow.log_dict(dataset.get_class_mapping(), "class_mapping.json")

    # Train and Evaluate the Model
    train_model(model, train_loader,val_loader, optimizer, criterion, NUM_EPOCHS, DEVICE, 
                save_checkpoint_interval=10, checkpoint_dir=CHECKPOINTS_PATH, 
                model_name=model_name, early_stopping_patience=10, cnn=False)
    _,_ = evaluate_model(model, test_loader, dataset, DEVICE, 
                         results_dir=results_dir,
                         img_path='../plots', 
                         run_name=run_name,
                         cnn=False)

    # Log the model
    mlflow.pytorch.log_model(model, model_name)

2024/06/22 18:54:13 INFO mlflow.system_metrics.system_metrics_monitor: Started monitoring system metrics.


Epoch 1/50:   0%|          | 0/350 [00:00<?, ?batch/s]

Epoch 2/50:   0%|          | 0/350 [00:00<?, ?batch/s]

Epoch 3/50:   0%|          | 0/350 [00:00<?, ?batch/s]

2024/06/22 19:06:38 INFO mlflow.system_metrics.system_metrics_monitor: Stopping system metrics monitoring...
2024/06/22 19:06:38 INFO mlflow.system_metrics.system_metrics_monitor: Successfully terminated system metrics monitoring!


KeyboardInterrupt: 